In [1]:
import pandas as pd
import numpy as np

In [22]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [2]:
df = pd.read_csv("data/train.csv")

In [3]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df = df.drop("id",axis=1)

In [5]:
X = df.drop("price",axis=1)
y = df["price"]

In [6]:
categorical_cols = X.select_dtypes(include="object").columns
numerical_cols = X.select_dtypes(exclude="object").columns

In [7]:
cut_categories = ["Fair","Good","Very Good","Premium","Ideal"]
color_categories = ["D","E","F","G","H","I","J"]
clarity_categories = ["I1","SI2","SI1","VS2","VS1","VVS2","VVS1","IF"]

In [9]:
num_pipeline = Pipeline(
    steps = [
        ("impute",SimpleImputer(strategy="median")),
        ("scaler",StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps= [
        ("impute",SimpleImputer(strategy="most_frequent")),
        ("oe",OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ("sc",StandardScaler())
    ]
)

preprocessor = ColumnTransformer([
    ("num_pipeline",num_pipeline,numerical_cols),
    ("cat_pipeline",cat_pipeline,categorical_cols)
])

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [13]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.fit_transform(X_test),columns=preprocessor.get_feature_names_out())

In [14]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-1.016395,-0.204317,0.402608,-1.202472,-1.187395,-1.194148,-0.132842,-0.936018,-0.648950
1,0.882396,0.720758,-0.118536,0.985177,0.941823,1.036109,-0.132842,-0.320002,0.017052
2,1.529711,0.350728,-1.160823,1.426308,1.394848,1.441611,0.872563,1.528047,0.017052
3,1.896523,0.073206,0.923751,1.741402,1.711965,1.702290,-0.132842,1.528047,-1.314953
4,0.450852,1.738340,1.444895,0.562052,0.525040,0.703019,-2.143651,0.912031,0.017052


In [18]:
reg = LinearRegression()
reg.fit(X_train,y_train)

LinearRegression()

In [19]:
reg.coef_

array([ 6421.41602296,  -129.81979941,   -68.93970393, -1772.23808191,
        -409.88800957,   -81.57634354,    70.26517377,  -463.94836356,
         652.41368541])

In [20]:
reg.intercept_

3972.01423239355

In [21]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    r2_square = r2_score(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    return mae,mse,rmse,r2_square

In [23]:
models = {
    "Linear Regression":LinearRegression(),
    "Ridge":Ridge(),
    "Lasso":Lasso(),
    "Elastic":ElasticNet()
}

trained_model = []
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)

    mae,mse,rmse,r2 = evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])

    print("Model Training Performance")
    print("RMSE",rmse)
    print("MAE",mae)
    print("MSE",mse)
    print("r2",r2)

    r2_list.append(r2)

    print("*"*30)
    print()

Linear Regression
Model Training Performance
RMSE 1006.5774449129776
MAE 674.0468954216653
MSE 1013198.1526075385
r2 0.9373004483372436
******************************

Ridge
Model Training Performance
RMSE 1006.5804399377092
MAE 674.0758284770371
MSE 1013204.1820651921
r2 0.937300075217444
******************************

Lasso
Model Training Performance
RMSE 1006.7930598833645
MAE 675.4194857587046
MSE 1013632.2654293082
r2 0.937273584214736
******************************

Elastic
Model Training Performance
RMSE 1527.7386434279647
MAE 1065.5872665074623
MSE 2333985.362623118
r2 0.855566420598686
******************************

